In [ ]:
# ============ IMPORTS ============
import pandas as pd
import numpy as np
import sys

# Add src folder to path
sys.path.insert(0, 'src')
from src.data_download import CACHE_FILE
from src.features import calculate_all_features
from src.scaling import scale_all_features

#  ============ AUTO-RELOAD UPDATED MODULES  ============ 
%load_ext autoreload
%autoreload 2

# ============ LOAD CACHED DATA ============
cache = pd.read_pickle(CACHE_FILE)
bitcoin_historical_data = cache['bitcoin_data']
fear_greed_index_data = cache['fgi_data']
start_date = cache['start_date']
end_date = cache['end_date']

print(f"Loaded from {cache['cache_date']}")

# ============ CALCULATE ALL FEATURES ============
# This contains BOTH raw OHLCV AND engineered features (including Cycle_Low_Multiple)
features_df = calculate_all_features(bitcoin_historical_data, fear_greed_index_data)

# ============ PREPARE DATA FOR DASHBOARD ============
# Separate raw OHLCV columns from engineered features
ohlcv_columns = [
    col for col in features_df.columns
    if isinstance(col, tuple) and len(col) == 2 and col[1] == 'BTC-USD'
]

# Extract only engineered features (no raw OHLCV) for scaling
engineered_features = features_df.drop(ohlcv_columns, axis=1, errors='ignore')

# Flatten MultiIndex if present
if isinstance(engineered_features.columns, pd.MultiIndex):
    engineered_features.columns = engineered_features.columns.get_level_values(0)

# Scale the engineered features
scaled_features = scale_all_features(engineered_features)

# ============ OUTPUT DATAFRAMES ============
bitcoin_data = features_df  # Keep original with OHLCV for price display
absolute_values = scaled_features['absolute_values']  # Scaled features (absolute)
normalised_values = scaled_features['normalised_values']  # Scaled features (0-1)
pd.to_pickle({
    'bitcoin_data': bitcoin_data,
    'absolute_values': absolute_values,
    'normalised_values': normalised_values
}, 'data_cache/dashboard_data.pkl')
